# Using Random Forest Classifiers to detect anomalies from FinBIF specimen data

Random Forest is recommended method by many scientific papers. It only evaluates a random subset of predictors to identify the best predictors instead of searching over all predictors. 

Down-sampling (‘balanced RF') is used in this method to reduce the bias of overlapping / imbalanced classes. See https://nsojournals.onlinelibrary.wiley.com/doi/full/10.1111/ecog.05615

In [1]:
import pandas as pd
import geopandas as gpd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import warnings
import helpers
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


warnings.filterwarnings('ignore') # Suppress warnings for cleaner output

In [2]:
"""
Read the data. The geopackage has 289 000 YKJ squares (10 km x 10 km grid polygons). 
Each of them contain:
    - the bird species in the square, and its atlas class value, ID and name
    - CORINE land cover proportions
    - temperature and height
    - YKJ coordinates (ykj_n and ykj_e)
    - the lenght of the coastline in the square

In finland, there are 3816 YKJ squares in total, so this geopackage has multiple overlapping polygons, one for each bird.
"""
gdf = gpd.read_file('YKJ10km_polygons\YKJ_Corine_birds2.gpkg')

In [17]:
# Define feature columns (environment, habitat and coordinates)
features = ['coastline', 'temp', 'dem', 'Urban', 'Park', 'Rural', 'Forest', 'Open_forest', 'Fjell', 'Open_area', 'Wetland', 'Open_bog', 'Freshwater', 'Marine', 'ykj_n', 'ykj_e']

In [18]:
# Split into present and absent data
gdf_present = gdf[
    (gdf['atlas_class_value'] == 'Varma pesintä') | 
    (gdf['atlas_class_value'] == 'Todennäköinen pesintä')
]

gdf_absent = gdf[
    (gdf['atlas_class_value'] == 'Epätodennäköinen pesintä') | 
    (gdf['atlas_class_value'] == 'Mahdollinen pesintä')
]



In [ ]:

print('Number of present data:', len(gdf_present))
print('Number of absent data:', len(gdf_absent))

In [ ]:
# Combine presence and absence data into a single GeoDataFrame. Add column 'class'
merged = helpers.stack_geodataframes(gdf_present, gdf_absent, add_class_label=True)
merged.describe()

In [21]:
# Load the YKJ grid square data with similar environmental features but without birds
gdf_new_grids = gpd.read_file('YKJ10km_polygons/YKJ_polygons_corine_no_birds.gpkg')

# Prepare the feature matrix for the new grid squares
X_new_grids = gdf_new_grids[features]

# Normalize feature columns
X_new_grids = helpers.normalise_columns(X_new_grids)

In [22]:
# Normalize all columns in the merged GeoDataFrame
merged = helpers.normalise_columns(merged)

In [27]:
def print_statistics(rf, y_test, rf_predictions, rf_probs):
    # Calculate accuracy
    accuracy = accuracy_score(y_test, rf_predictions)
    print(f'Accuracy: {accuracy:.4f}')

    # Calculate precision
    precision = precision_score(y_test, rf_predictions)
    print(f'Precision: {precision:.4f}')

    # Calculate recall
    recall = recall_score(y_test, rf_predictions)
    print(f'Recall: {recall:.4f}')

    # Calculate F1 score
    f1 = f1_score(y_test, rf_predictions)
    print(f'F1 Score: {f1:.4f}')

    # Calculate ROC-AUC score
    roc_auc = roc_auc_score(y_test, rf_probs)
    print(f'ROC-AUC Score: {roc_auc:.4f}')

    # Get feature importances
    feature_importances = rf.feature_importances_

    # Create a DataFrame to pair feature names with their importance scores
    importance_df = pd.DataFrame({
        'Feature': features, 
        'Importance': feature_importances
    })

    # Sort features by importance
    importance_df = importance_df.sort_values(by='Importance', ascending=False)

    # Display the top features
    print(importance_df)

    print('\n')

In [24]:
# Define hyperparameter grid for tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Define cross-validation strategy
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [ ]:
# Group by "species_name" and loop through each species
for species_name, species_data in merged.groupby('species_name'):
    if len(species_data) < 1000:
        continue # Skip species with less than 1000 data points
    
    print('Species:', species_name)

    # Prepare feature matrices and target variables
    X = species_data[features]
    y = species_data['class']
    weights = species_data['weights']

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(X, y, weights, test_size=0.3, random_state=42)

    # Perform grid search with cross-validation
    rf = RandomForestClassifier(random_state=42)
    grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=cv, scoring='roc_auc', n_jobs=-1)
    grid_search.fit(X_train, y_train, sample_weight=weights_train)

    # Get the best model from grid search
    best_rf = grid_search.best_estimator_

    rf_predictions = best_rf.predict(X_test)
    rf_probs = best_rf.predict_proba(X_test)[:, 1]

    #print_statistics(best_rf, y_test, rf_predictions, rf_probs)
    
    # Predict probabilities using the trained Random Forest model
    gdf_new_grids['probability'] = best_rf.predict_proba(X_new_grids)[:, 1]

    # Save the results to a new file
    #gdf_new_grids.to_file(f'data_results/geopackages/{species_name}_predictions.gpkg', driver='GPKG')

    # Plot the results on a map
    fig, ax = plt.subplots(figsize=(12, 12))
    gdf_new_grids.plot(column='probability', cmap='coolwarm', legend=True, ax=ax)
    plt.title(f'{species_name} Probability Map')
    #plt.show()

    # Store results as png
    plt.savefig(f'data_results/images/{species_name}_probability_map3.png')
    plt.close()
